### Baselines:
- BERT-based classifier trained on the data

### Ideas:
- Using NLI model: https://huggingface.co/microsoft/deberta-base-mnli and fine-tuning it on the data we have.
- Validity on the premise (sentence) level.

In [1]:
import wandb
wandb.init(project="argsvalidnovel")

wandb: Currently logged in as: miladalsh. Use `wandb login --relogin` to force relogin


In [2]:
%load_ext autoreload

In [3]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [4]:
%autoreload
import sbert_training
from utils import *

In [5]:
from datasets import load_dataset, load_metric, Dataset, Split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, DebertaForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support


In [6]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/"

In [7]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_valid_df = taska_valid_df[taska_valid_df.Validity != 0]

In [8]:
nli_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
nli_model     = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')
arg_stance_pipeline = TextClassificationPipeline(model=nli_model, tokenizer=nli_tokenizer, framework='pt', task='validity_classifier', device=0)

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
#without topic
taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP]'.format(x['Premise'], x['Conclusion']), axis=1)

resutls = arg_stance_pipeline(taska_valid_df['input_txt'].tolist())
taska_valid_df['pred_label'] = [1 if x['label'] == 'ENTAILMENT' else -1 for x in resutls]

precision, recall, f1, _ = precision_recall_fscore_support(taska_valid_df.Validity.tolist(), taska_valid_df.pred_label.tolist(), average='binary')

print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))

Precision: 0.8041237113402062, Recall 0.624, F1: 0.7027027027027027


In [13]:
#with topic
taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

resutls = arg_stance_pipeline(taska_valid_df['input_txt'].tolist())
taska_valid_df['pred_label'] = [1 if x['label'] == 'ENTAILMENT' else -1 for x in resutls]

precision, recall, f1, _ = precision_recall_fscore_support(taska_valid_df.Validity.tolist(), taska_valid_df.pred_label.tolist(), average='binary')

print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))

Precision: 0.8017241379310345, Recall 0.744, F1: 0.7717842323651453


So with topic is better...

In [9]:
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

## Fine-tune the NLI model on the training data:

In [10]:
nli_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
nli_model     = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
train_dataset = Dataset.from_pandas(taska_training_df)
eval_dataset = Dataset.from_pandas(taska_valid_df)

In [12]:
nli_model.config.id2label

{0: 'CONTRADICTION', 1: 'NEUTRAL', 2: 'ENTAILMENT'}

In [13]:
validity_map = dict([
    (1, 2), # if valid -> entailment label
    (-1, 0) # not valid -> contradiction (neutral) label
])

In [14]:
inverse_validity_map = dict([
    (2,1),
    (1,-1),
    (0,-1) 
])

In [15]:
def preprocess(example):
    inputs = nli_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, max_length=512)
    inputs['labels'] = list(map(validity_map.get, example['Validity']))
    return inputs

In [16]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

2022-07-05 14:16:34 - Parameter 'function'=<function preprocess at 0x7f9e674860d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
training_args = TrainingArguments(
    output_dir= output_path + "/task-A/validity/classification/nli-model",
    report_to="wandb",
    overwrite_output_dir=True,
    metric_for_best_model = 'f1',
    evaluation_strategy = 'steps',  
    learning_rate = 5e-5,                   # we can customize learning rate
    max_steps = 200, # five epochs
    logging_steps = 50,                    # we will log every 50 steps which is an epoch given the 700 examples and 16 batch size
    eval_steps = 50,                      # we will perform evaluation every 500 steps
    save_steps = 50,
    load_best_model_at_end = True,
)

In [20]:
trainer = Trainer(
    model=nli_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda x: compute_nli_metrics(x, average='binary')
)

max_steps is given, it will override any value given in num_train_epochs


In [21]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Novelty-Confidence, __index_level_0__, Validity, topic, Conclusion, input_txt, Premise, Validity-Confidence, Novelty. If Novelty-Confidence, __index_level_0__, Validity, topic, Conclusion, input_txt, Premise, Validity-Confidence, Novelty are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 1.787210464477539,
 'eval_recall': 0.72,
 'eval_precision': 0.7894736842105263,
 'eval_f1': 0.7531380753138075,
 'eval_runtime': 4.203,
 'eval_samples_per_second': 47.348,
 'eval_steps_per_second': 3.093}

In [22]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Novelty-Confidence, __index_level_0__, Validity, topic, Conclusion, input_txt, Premise, Validity-Confidence, Novelty. If Novelty-Confidence, __index_level_0__, Validity, topic, Conclusion, input_txt, Premise, Validity-Confidence, Novelty are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 721
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation 

Step,Training Loss,Validation Loss,Recall,Precision,F1
50,0.565300,0.622428,0.992000,0.696629,0.818482
100,0.266000,1.020044,0.992000,0.700565,0.821192
150,0.107200,1.574849,0.992000,0.708571,0.826667
200,0.028400,1.687001,0.992000,0.729412,0.840678


The following columns in the evaluation set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Novelty-Confidence, __index_level_0__, Validity, topic, Conclusion, input_txt, Premise, Validity-Confidence, Novelty. If Novelty-Confidence, __index_level_0__, Validity, topic, Conclusion, input_txt, Premise, Validity-Confidence, Novelty are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8
Saving model checkpoint to ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/validity/classification/nli-model/checkpoint-50
Configuration saved in ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/validity/classification/nli-model/checkpoint-50/config.json
Model weights saved in ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/validity/classification/nli-model/checkpoint-50/pytorch_model.bin
/usr/loca

TrainOutput(global_step=200, training_loss=0.24172223269939422, metrics={'train_runtime': 180.0213, 'train_samples_per_second': 17.776, 'train_steps_per_second': 1.111, 'total_flos': 233159689548000.0, 'train_loss': 0.24172223269939422, 'epoch': 4.35})

In [28]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion. If Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.5748487710952759,
 'eval_recall': 0.6514054054054054,
 'eval_precision': 0.833452380952381,
 'eval_f1': 0.6480272108843537,
 'eval_runtime': 0.5444,
 'eval_samples_per_second': 365.545,
 'eval_steps_per_second': 23.88,
 'epoch': 4.35}

## Fine-tune simple BERT model on the training data:

In [29]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model     = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://hugg

In [30]:
train_dataset = Dataset.from_pandas(taska_training_df)
eval_dataset = Dataset.from_pandas(taska_valid_df)

In [31]:
validity_map = dict([ # avoid negative labels
    (1, 1), 
    (-1, 0)
])

In [32]:
def preprocess(example):
    inputs = bert_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, truncation=True, max_length=512)
    inputs['labels'] = list(map(validity_map.get, example['Validity']))
    return inputs

In [33]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [34]:
training_args = TrainingArguments(
    output_dir= output_path + "/task-A/validity/classification/bert", 
    report_to="wandb",
    overwrite_output_dir=True,
    metric_for_best_model = 'f1',
    evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    learning_rate = 5e-5,                   # we can customize learning rate
    max_steps = 200, # five epochs
    logging_steps = 50,                    # we will log every 50 steps which is an epoch given the 700 examples and 16 batch size
    eval_steps = 50,                      # we will perform evaluation every 500 steps
    save_steps = 50,
    load_best_model_at_end = True,
)

PyTorch: setting up devices


In [35]:
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda x: compute_metrics(x, average='macro')
)

max_steps is given, it will override any value given in num_train_epochs


In [36]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion. If Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 721
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps 

Step,Training Loss,Validation Loss,Recall,Precision,F1
50,0.674400,0.598687,0.520541,0.591876,0.453975
100,0.454900,0.608931,0.663514,0.675058,0.667224
150,0.338300,0.742523,0.591405,0.635497,0.585996
200,0.196400,0.825659,0.629459,0.658651,0.632615


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion. If Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8
Saving model checkpoint to ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/validity/classification/bert/checkpoint-50
Configuration saved in ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/validity/classification/bert/checkpoint-50/config.json
Model weights saved in ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/validity/classification/bert/checkpoint-50/pytorch_model.bin
/usr/local/lib/python3.8/dist-

TrainOutput(global_step=200, training_loss=0.41601430416107177, metrics={'train_runtime': 140.2051, 'train_samples_per_second': 22.824, 'train_steps_per_second': 1.426, 'total_flos': 200087735380800.0, 'train_loss': 0.41601430416107177, 'epoch': 4.35})

In [37]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion. If Validity, __index_level_0__, input_txt, Novelty, Novelty-Confidence, Validity-Confidence, Premise, topic, Conclusion are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.6089308261871338,
 'eval_recall': 0.6635135135135135,
 'eval_precision': 0.6750578703703703,
 'eval_f1': 0.6672240802675584,
 'eval_runtime': 0.4304,
 'eval_samples_per_second': 462.411,
 'eval_steps_per_second': 30.208,
 'epoch': 4.35}